# Week 2 — Linear Regression 2 (CKD)

This notebook applies Linear Regression (OLS, Ridge, Lasso, ElasticNet) to the CKD dataset and explores degree-2 polynomial features.

**Target:** `SerumCreatinine` (change if needed)


## 1) Setup & Data Load

In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# If running locally, put the CSV in the same folder as this notebook
DATA_PATH = 'Chronic_Kidney_Dsease_data.csv'
TARGET_COL = 'SerumCreatinine'  # change if needed

# Load
df = pd.read_csv(DATA_PATH)
df.columns = [str(c).strip() for c in df.columns]

def coerce_numeric(s):
    return pd.to_numeric(s.replace({'?': np.nan, 'NA': np.nan, 'None': np.nan, 'na': np.nan, '': np.nan}), errors='coerce')

# Convert object columns that are mostly numeric-looking
for c in df.columns:
    if df[c].dtype == object:
        z = coerce_numeric(df[c])
        if np.isfinite(z).mean() > 0.6:
            df[c] = z

# Split features/target
assert TARGET_COL in df.columns, f"'{TARGET_COL}' not found in dataframe."
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Column typing
num_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
cat_cols = [c for c in X.columns if c not in num_cols]

# Preprocessing
numeric_pre = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorical_pre = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
])

pre = ColumnTransformer([
    ('num', numeric_pre, num_cols),
    ('cat', categorical_pre, cat_cols),
])

# CV & metrics
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'rmse': 'neg_root_mean_squared_error',
    'mae' : 'neg_mean_absolute_error',
    'r2'  : 'r2'
}

print('Shape:', df.shape)
df.head(8)

## 2) Baselines: OLS, Ridge, Lasso, ElasticNet (5-fold CV)

In [ ]:
import numpy as np

results = {}

def eval_model(name, model):
    pipe = Pipeline([('pre', pre), ('model', model)])
    cvres = cross_validate(pipe, X, y, scoring=scoring, cv=cv, return_train_score=False)
    results[name] = {
        'RMSE_mean': -cvres['test_rmse'].mean(),
        'RMSE_std' :  cvres['test_rmse'].std(),
        'MAE_mean' : -cvres['test_mae'].mean(),
        'MAE_std'  :  cvres['test_mae'].std(),
        'R2_mean'  :  cvres['test_r2'].mean(),
        'R2_std'   :  cvres['test_r2'].std(),
    }

ridge_alphas = np.logspace(-3, 3, 25)

eval_model('LinearRegression', LinearRegression())
eval_model('RidgeCV', RidgeCV(alphas=ridge_alphas))
eval_model('LassoCV', LassoCV(cv=cv, random_state=42, max_iter=10000))
eval_model('ElasticNetCV', ElasticNetCV(l1_ratio=[.2,.4,.6,.8,.9,.95,.99,1.0], cv=cv, random_state=42, max_iter=10000))

baseline_df = pd.DataFrame(results).T.sort_values('RMSE_mean')
baseline_df.round(4)

## 3) Polynomial Features (degree 2, numeric only)

In [ ]:
numeric_poly = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

pre_poly = ColumnTransformer([
    ('num', numeric_poly, num_cols),
    ('cat', categorical_pre, cat_cols),
])

poly_results = {}

def eval_model_poly(name, model):
    pipe = Pipeline([('pre', pre_poly), ('model', model)])
    cvres = cross_validate(pipe, X, y, scoring=scoring, cv=cv, return_train_score=False)
    poly_results[name] = {
        'RMSE_mean': -cvres['test_rmse'].mean(),
        'RMSE_std' :  cvres['test_rmse'].std(),
        'MAE_mean' : -cvres['test_mae'].mean(),
        'MAE_std'  :  cvres['test_mae'].std(),
        'R2_mean'  :  cvres['test_r2'].mean(),
        'R2_std'   :  cvres['test_r2'].std(),
    }

# Run a few representative polynomial variants
eval_model_poly('LinearRegression+Poly2', LinearRegression())
eval_model_poly('RidgeCV+Poly2', RidgeCV(alphas=ridge_alphas))
eval_model_poly('LassoCV+Poly2', LassoCV(cv=cv, random_state=42, max_iter=10000))
eval_model_poly('ElasticNetCV+Poly2', ElasticNetCV(l1_ratio=[.2,.4,.6,.8,.9,.95,.99,1.0], cv=cv, random_state=42, max_iter=10000))

poly_df = pd.DataFrame(poly_results).T.sort_values('RMSE_mean')
poly_df.round(4)

combined = pd.concat([baseline_df, poly_df]).sort_values('RMSE_mean')
combined.round(4)

## 4) Holdout & Residual Diagnostics

In [ ]:
best_name = combined.index[0]
use_poly = 'Poly2' in best_name

if use_poly:
    use_pre = pre_poly
else:
    use_pre = pre

if 'Ridge' in best_name:
    best_est = RidgeCV(alphas=ridge_alphas)
elif 'Lasso' in best_name:
    best_est = LassoCV(cv=cv, random_state=42, max_iter=10000)
elif 'ElasticNet' in best_name:
    best_est = ElasticNetCV(l1_ratio=[.2,.4,.6,.8,.9,.95,.99,1.0], cv=cv, random_state=42, max_iter=10000)
else:
    best_est = LinearRegression()

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=7)
pipe_best = Pipeline([('pre', use_pre), ('model', best_est)]).fit(X_tr, y_tr)
y_hat = pipe_best.predict(X_te)

rmse = mean_squared_error(y_te, y_hat, squared=False)
mae  = mean_absolute_error(y_te, y_hat)
r2   = r2_score(y_te, y_hat)

print('Best by CV:', best_name)
print(f'Holdout RMSE: {rmse:.4f}')
print(f'Holdout MAE : {mae:.4f}')
print(f'Holdout R^2 : {r2:.4f}')

# Residuals vs Fitted
resid = y_te - y_hat
plt.figure()
plt.scatter(y_hat, resid, alpha=0.7)
plt.axhline(0, linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted — ' + best_name)
plt.show()

# Simple Q–Q without seaborn/scipy
res_sorted = np.sort(resid)
n = len(res_sorted)
probs = (np.arange(1, n+1) - 0.5) / n
theoretical = np.sqrt(2) * np.erfinv(2*probs - 1)
z_res = (res_sorted - res_sorted.mean()) / res_sorted.std(ddof=1)

plt.figure()
plt.scatter(theoretical, z_res, alpha=0.7)
mn, mx = theoretical.min(), theoretical.max()
plt.plot([mn, mx], [mn, mx], linestyle='--')
plt.xlabel('Theoretical Quantiles (Z)')
plt.ylabel('Standardized Residual Quantiles')
plt.title('Q–Q Plot — ' + best_name)
plt.show()

## 5) Notes (edit this block)

- **Best model (by CV RMSE):** _(fill from `combined` table)_  
- **Holdout metrics:** RMSE≈…, MAE≈…, R²≈…  
- **Polynomial features:** Helped/hurt depending on interaction strength; regularization controlled variance.  
- **Assumptions:** Residuals vs fitted suggest _[approx/non]_ constant variance; Q–Q suggests _[approx/non]_ normality.  
- **Next steps:** Try log transforms for skewed labs, targeted interactions, and compare to tree-based models in Week 6.  
